Erste Modelle testen auf die raw Daten.


In [ ]:
import pandas as pd
import matplotlib as plt
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
from statsmodels.tsa.arima.model import ARIMA
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression


In [4]:
df = pd.read_csv("../data/preprocessed/Locations_00_prep.csv")
df.columns = df.columns.str.lower()
df_1 = df.loc[df["day"]==1]
#print(df.iloc[210:221])

## Linear

In [9]:
# Very Simple Linear Regression Model
# Extract data
x = df_1[['day', 'time']]
y = df_1[['x', 'y', 'z']]

# Create a linear regression model and fit it to the data
model = LinearRegression()
model.fit(x, y)

# Make a prediction for a given time x
x_test = np.array([[1,8.903437]])  # Replace with the time for which you want to make a prediction
y_pred = model.predict(x_test)

# Print the predicted values
print(y_pred)  # This will output an array with the predicted values for y1, y2, and y3 for the given time x_test


[[ 0.70760237 -2.39520294 -9.26549638]]


c:\Users\Philipp\anaconda3\envs\YoloTraining\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## Random Forest

In [ ]:
x = df[['day', 'time']]
y = df[['x', 'y', 'z']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=None)

# Train a random forest regression model
rf = RandomForestRegressor(n_estimators=8, random_state=42)
rf.fit(X_train, y_train)

# Make a prediction for a new data point
new_data = [[1, 16.7]]  # A new data point with day=1 and time=9.5
prediction = rf.predict(new_data)

print(prediction)  # Prints the predicted (x, y, z) values


In [5]:
df["hour"] = df["time"].astype(int)
df["minute"] = (60 * (df["time"]-df["hour"])).astype(int)

# Convert the columns to a common data type
df['hour'] = df['hour'].astype('int')
df['minute'] = df['minute'].astype('int')/60

# Combine the columns into a single string
df['time_new'] = df['hour'] +  df['minute']
df.head(5)

,day,time,time_new,day_new,week_num,x,y,z,no_movement,hour,minute
0,1,8.900010,8.9,1,0.0,-0.410000,-7.217000,-8.120000,NaN,8,0.9
1,1,8.901722,8.9,1,0.0,-0.912195,-6.359423,-9.059900,NaN,8,0.9
2,1,8.903437,8.9,1,0.0,-2.666780,-6.366762,-12.617724,NaN,8,0.9
3,1,8.905137,8.9,1,0.0,-1.125336,-6.381155,-15.999400,NaN,8,0.9
4,1,8.906867,8.9,1,0.0,1.372597,-6.392756,-18.155860,NaN,8,0.9


In [12]:
df = pd.read_csv("../data/preprocessed/Locations_00_prep.csv")
df.columns = df.columns.str.lower() 
df = df.loc[df["day_new"]==1]
#df = df.loc[df["no_movement"]==True]
print(len(df))
df.head()

16122


,day,time,time_new,day_new,week_num,x,y,z,no_movement
0,1,8.900010,8:54:0,1,0.0,-0.410000,-7.217000,-8.120000,NaN
1,1,8.901722,8:54:6,1,0.0,-0.912195,-6.359423,-9.059900,NaN
2,1,8.903437,8:54:12,1,0.0,-2.666780,-6.366762,-12.617724,NaN
3,1,8.905137,8:54:18,1,0.0,-1.125336,-6.381155,-15.999400,NaN
4,1,8.906867,8:54:24,1,0.0,1.372597,-6.392756,-18.155860,NaN


In [13]:
df["hour"] = df["time"].astype(int)
df["minute"] = (60 * (df["time"]-df["hour"])).astype(int)

# Convert the columns to a common data type
df['hour'] = df['hour'].astype('int')
df['minute'] = df['minute'].astype('int')/60

# Combine the columns into a single string
df['time_new'] = df['hour'] +  df['minute']
df.head(5)

,day,time,time_new,day_new,week_num,x,y,z,no_movement,hour,minute
0,1,8.900010,8.9,1,0.0,-0.410000,-7.217000,-8.120000,NaN,8,0.9
1,1,8.901722,8.9,1,0.0,-0.912195,-6.359423,-9.059900,NaN,8,0.9
2,1,8.903437,8.9,1,0.0,-2.666780,-6.366762,-12.617724,NaN,8,0.9
3,1,8.905137,8.9,1,0.0,-1.125336,-6.381155,-15.999400,NaN,8,0.9
4,1,8.906867,8.9,1,0.0,1.372597,-6.392756,-18.155860,NaN,8,0.9


In [ ]:
x = df[['day_new', 'time_new']]
y = df[['x', 'y', 'z']]

# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

# Split training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, shuffle=False)

best_mse = 2000
best_estimators = 0

for i in range(1,100,20):
    # Train a random forest regression model
    rf = RandomForestRegressor(n_estimators=i)
    rf.fit(X_train, y_train)

    # Evaluate the model on the test set using RMSE
    y_pred = rf.predict(X_test)
    mse_new = mean_absolute_error(y_test, y_pred)


    if mse_new < best_mse:
        best_mse = mse_new
        best_estimators = i

    print(i)

print("best_estimators: "+ str(best_estimators))
print("best_mse: "+ str(best_mse))
print("best_rmse: "+ str(math.sqrt(best_mse)))


# Make a prediction for a new data point
new_data = [[1, 9]]  # A new data point with day=1 and time=9.5
prediction = rf.predict(new_data)

print(str(new_data) +": " + str(prediction))  # Prints the predicted (x, y, z) values

In [33]:
# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Split training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

best_rmse = 20
best_estimators = 0

i = 66
best_mse = 20
best_estimators = 0
# Train a random forest regression model
rf = RandomForestRegressor(n_estimators=i, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model on the test set using RMSE
y_pred = rf.predict(X_test)
mse_new = mean_squared_error(y_test, y_pred)


if rmse_new < best_mse:
    best_mse = mse_new
    best_estimators = i

print("best_estimators: "+ str(best_estimators))
print("best_mse: "+ str(best_mse))
print("best_rmse: "+ str(math.sqrt(best_mse)))


# Make a prediction for a new data point
new_data = [[1, 9]]  # A new data point with day=1 and time=9.5
prediction = rf.predict(new_data)

print(prediction)  # Prints the predicted (x, y, z) values

best_estimators: 66
best_mse: 6.872106971325752
best_rmse: 2.6214703834538646
[[11.0779421  -0.2594229   1.75234293]]


c:\Users\Philipp\anaconda3\envs\YoloTraining\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


## SVM

In [14]:
df = pd.read_csv("../data/preprocessed/Locations_00_prep.csv")
df.columns = df.columns.str.lower()
df_1 = df.loc[df["day"].isin([0,1,2,3,4,5,6,7,8,9,10])]


x = df_1[['day', 'time']]
y = df_1[['x', 'y', 'z']]

In [15]:
# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Split training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Reshape X_train to have shape (n_samples, n_features)
X_train = np.array(X_train)
X_train = X_train.reshape((-1, 2))  # Assumes that each sample has two features (day and time)

best_rmse = 20
best_estimators = 0

best_mse = 20
best_C = 0
best_kernel = 'linear'

# Create a multi-output SVR model
svm = SVR(kernel='linear', C=1)
multi_svm = MultiOutputRegressor(svm, n_jobs=-1)

# Train a support vector machine regression model
for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
    for C in [0.1, 1, 10]:
        svm = SVR(kernel=kernel, C=C)
        multi_svm.fit(X_train, y_train)

        # Evaluate the model on the validation set using RMSE
        y_pred = multi_svm.predict(X_val)
        mse_new = mean_squared_error(y_val, y_pred)
        rmse_new = np.sqrt(mse_new)

        if rmse_new < best_rmse:
            best_rmse = rmse_new
            best_mse = mse_new
            best_C = C
            best_kernel = kernel

print("best_rmse: "+ str(best_rmse))
print("best_mse: "+ str(best_mse))
print("best_kernel: "+ str(best_kernel))
print("best_C: "+ str(best_C))
 
# Train the final SVM model on the entire training set using the best hyperparameters
final_svm = SVR(kernel=best_kernel, C=best_C)
final_svm = MultiOutputRegressor(final_svm, n_jobs=-1)
final_svm.fit(X_train_val, y_train_val)

# Evaluate the final model on the test set using MSE
y_pred = final_svm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

# Make a prediction for a new data point
new_data = [[1, 16.6]]  # A new data point with day=1 and time=9.5
prediction = final_svm.predict(new_data)

print(prediction)  # Prints the predicted value



best_rmse: 6.378486475176701
best_mse: 40.68508971401209
best_kernel: linear
best_C: 0.1
MSE: 40.03642236645336
RMSE: 6.32743410605384
[[ 6.30602555 -0.35943901 -6.75284632]]


In [4]:
df = pd.read_csv("../data/preprocessed/Locations_00_prep.csv")
df.columns = df.columns.str.lower()
df_1 = df.loc[df["day"].isin([0,1,2,3,4,5,6,7,8,9,10])]


x = df_1[['day', 'time']]
y = df_1[['x', 'y', 'z']]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error

# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Split training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Define the Gaussian process regression model
kernel = RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.1)

# Train the model on the training set
gpr.fit(X_train, y_train)

# Predict on the validation set
y_pred = gpr.predict(X_val)

# Evaluate the model on the validation set using RMSE
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.3f}")


c:\Users\flipj\miniconda3\envs\myenv\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


RMSE: 7.888


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the model architecture
model = Sequential()
model.add(LSTM(units=64, input_shape=(timesteps, input_dim)))
model.add(Dense(units=3, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


## LSTM Model

In [9]:
df = pd.read_csv("../data/preprocessed/Locations_00_prep.csv")
df.columns = df.columns.str.lower()
df_1 = df.loc[df["day"].isin([1])]


x = df_1[['day', 'time']]
y = df_1[['x', 'y', 'z']]

# Split data into training and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Split training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

In [28]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        y_pred = self.fc(lstm_out[:, -1, ])
         # Add an extra dimension to y_pred
        y_pred = y_pred.unsqueeze(1)
        return y_pred



In [ ]:
model = LSTMPredictor(input_dim=2, hidden_dim=64, output_dim=3)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = torch.tensor(X_train.to_numpy()[i:i+batch_size], dtype=torch.float32)
        labels = torch.tensor(y_train.to_numpy()[i:i+batch_size], dtype=torch.float32)

        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate on the validation set
    with torch.no_grad():
        inputs = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
        labels = torch.tensor(y_val.to_numpy(), dtype=torch.float32)
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)
        print(f"Epoch {epoch+1}/{num_epochs}, train_loss: {loss.item()}, val_loss: {val_loss.item()}")
